In [4]:
import time
import re
import datetime
import hashlib
import pytz
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

options = Options()
options.add_argument('--headless') # ヘッドレスモードを有効にする

driver = webdriver.Chrome(chrome_options=options)
driver.set_page_load_timeout(10) #ページロードのタイムアウト(秒)

driver_wait = WebDriverWait(driver, 10) #操作した時のタイムアウト時間

# AT-Xの番組紹介ページから放送日を取得する

driver.get('https://www.at-x.com/encount/')
driver_wait.until(ec.presence_of_all_elements_located) #処理待ち

assert '東京エンカウント' in driver.title

print(driver.current_url)
data = driver.page_source.encode('utf-8')
soup = BeautifulSoup(data, 'lxml')
print(soup.title.string)

# ---------------

schedule = soup.find(id="schedule")

ttl_lv03 = schedule.find_all(class_='ttl_lv03')
time_table = schedule.find_all(class_='time_table')

def get_year_from_tag(tag):
    m = re.search('(\d{4})年', tag.text)
    assert m != None
    return int(m.group(1))
def get_day_from_tag(tag):
    m = re.search('(\d{1,2})月\D*(\d{1,2})日\D*(\d{1,2})\：(\d{1,2})', tag.text)
    assert m != None
    mo = int(m.group(1))
    day = int(m.group(2))
    h = int(m.group(3))
    m = int(m.group(4))
    return (mo, day, h, m)
def create_datetime(yyyy, mo, day, h, m, timezone):
    dt = datetime.datetime(yyyy, mo, day, h, m)
    tz = pytz.timezone(timezone)
    date = tz.localize(dt)
    return date

schedule_list = []
fdf_date = []
fdf_storynumber = []
for h3, div in zip(ttl_lv03, time_table):
    story_number = h3.find(class_='icon_number')
    year = div.find_all(class_='year')[0]
    days = div.find_all(class_='days')[0].find_all('li')
    for day in days:
        yyyy = get_year_from_tag(year)
        mo, day, h, m = get_day_from_tag(day)
        date = create_datetime(yyyy, mo, day, h, m, 'Asia/Tokyo')
        schedule_list.append((len(schedule_list), date.timestamp(), story_number.text))
        fdf_date.append(date)
        fdf_storynumber.append(story_number.text)

schedule_list.sort(key=lambda x:x[1]) #放送日順
print(schedule_list)

# ---------------

new_story_title = soup.find(id='right_contents').find(class_='ttl_lv02').text.strip()
new_story_body = soup.find(id='right_contents').find(class_='contents').text.strip()

print(new_story_title)
print(new_story_body)

# ---------------

schedule_digest = hashlib.sha224(str(schedule.text).encode('utf-8')).hexdigest()
print('schedule_digest : ', schedule_digest)


# # 検索語を入力して送信する。
# input_element = driver.find_element_by_name('q')
# input_element.send_keys('Python')
# input_element.send_keys(Keys.RETURN)

# time.sleep(2)  # Chromeの場合はAjaxで遷移するので、とりあえず適当に2秒待つ。

# # タイトルに'Python'が含まれていることを確認する。
# assert 'Python' in driver.title

# # スクリーンショットを撮る。
# driver.save_screenshot('search_results.png')

# # 検索結果を表示する。
# for a in driver.find_elements_by_css_selector('h3 > a'):
#     print(a.text)
#     print(a.get_attribute('href'))

driver.quit()  # ブラウザーを終了する。

https://www.at-x.com/encount/
東京エンカウント弐 | 番組 | AT-X ワンランク上のアニメ専門チャンネル
[(0, 1527946200.0, '第57章'), (1, 1528176600.0, '第57章'), (2, 1528407000.0, '第57章'), (3, 1528551000.0, '第57章'), (4, 1528781400.0, '第57章'), (5, 1529011800.0, '第57章'), (6, 1529155800.0, '第58章'), (7, 1529386200.0, '第58章'), (8, 1529616600.0, '第58章'), (9, 1529760600.0, '第58章'), (10, 1529991000.0, '第58章'), (11, 1530221400.0, '第58章')]
第五十七章の予告
今回も素敵なゲストにお越しいただきます。

なんと今回のゲストは...


小野大輔さんです！

小野さんをお迎えしてお送りするのは、
ちょっと久しぶりのアーケードエンカウント。
果たしてどんなタイトルが登場するのか。

お見逃しなく！
schedule_digest :  c2ec332236f4fe8f0de23885cfc2dd1306207038c58b8d1d2028434e


In [5]:
import pandas as pd
from datetime import datetime as dt

df = pd.DataFrame({
    'timestamp': [ date.timestamp() for date in fdf_date ],
    'date': [ date.strftime('%Y-%m-%d %H:%M:%S') for date in fdf_date ],
    'subject': fdf_storynumber
})
df

,date,subject,timestamp
0,2018-06-02 22:30:00,第57章,1.527946e+09
1,2018-06-05 14:30:00,第57章,1.528177e+09
2,2018-06-08 06:30:00,第57章,1.528407e+09
3,2018-06-09 22:30:00,第57章,1.528551e+09
4,2018-06-12 14:30:00,第57章,1.528781e+09
5,2018-06-15 06:30:00,第57章,1.529012e+09
6,2018-06-16 22:30:00,第58章,1.529156e+09
7,2018-06-19 14:30:00,第58章,1.529386e+09
8,2018-06-22 06:30:00,第58章,1.529617e+09
9,2018-06-23 22:30:00,第58章,1.529761e+09


## Install kiilib

```sh
$ pip3 install --user git+https://github.com/fkmhrk/kiilib_python.git
Collecting git+https://github.com/fkmhrk/kiilib_python.git
  Cloning https://github.com/fkmhrk/kiilib_python.git to /private/var/folders/ty/tnmd__m11vlg9hcjms4csxp00000gn/T/pip-7i035lhe-build
Installing collected packages: kiilib
  Running setup.py install for kiilib ... done
Successfully installed kiilib-0.1.dev0
```

In [4]:
import json
import kiilib

config = json.load(open('../config.json', 'r'))
print(config)

ModuleNotFoundError: No module named 'kii'

In [46]:
import re
import datetime
import pytz
import hashlib

year = '\n        2018年'
print(type(year))

m = re.search('(\d{4})年', year)
yyyy = int(m.group(1))

day = '\n      2月 3日（土）22：30'
m = re.search('(\d{1,2})月\D*(\d{1,2})日\D*(\d{1,2})\：(\d{1,2})', day)
mo = int(m.group(1))
day = int(m.group(2))
h = int(m.group(3))
m = int(m.group(4))

# date = datetime.datetime(yyyy, mo, day, h, m, tzinfo=pytz.timezone('Asia/Tokyo'))
dt = datetime.datetime(yyyy, mo, day, h, m)
tz = pytz.timezone('Asia/Tokyo')
date = tz.localize(dt)
print(date, date.timestamp())

"""
(2018, 2, 3, 22, 30), 1517663460.0

GMT: 2018年2月3日 13:30:00 Saturday
Your time zone: 2018年2月3日 22:30:00 土曜日 GMT+09:00
"""

story = '今回も新たなゲストがやってきます！'
schedule_digest = hashlib.sha224(str(story).encode('utf-8')).hexdigest()
print('schedule_digest : ', schedule_digest)

<class 'str'>
2018-02-03 22:30:00+09:00 1517664600.0
schedule_digest :  ca7fc3dde39ea1400949ec7a507ab40af2748d393732c7b6375bc585
